# Simple LSTM with MNIST

In [1]:
try:
    from cloudmesh.common.StopWatch import StopWatch
except:  # noqa: E722
    ! pip install cloudmesh-common
    from cloudmesh.common.StopWatch import StopWatch

# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
StopWatch.start('total')
StopWatch.start('import')
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN, InputLayer, LSTM
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
from cloudmesh.common.StopWatch import StopWatch
StopWatch.stop('import')
StopWatch.progress(0)

# cloudmesh status=running progress=0 pid=3513


## Data Pre-Process

In [2]:
StopWatch.start("data-load")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
StopWatch.stop("data-load")


StopWatch.start("data-pre-process")
num_labels = len(np.unique(y_train))


y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size])
x_test = np.reshape(x_test,[-1, image_size, image_size])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
StopWatch.stop("data-pre-process")

input_shape = (image_size, image_size)
batch_size = 128
units = 256
dropout = 0.2
StopWatch.progress(20)

# cloudmesh status=running progress=20 pid=3513


## Define Model

In [3]:
StopWatch.progress(40)
StopWatch.start("compile")
model = Sequential()
model.add(LSTM(units=units,                      
                     input_shape=input_shape,
                     return_sequences=True))
model.add(LSTM(units=units, 
                     dropout=dropout,                      
                     return_sequences=True))
model.add(LSTM(units=units, 
                     dropout=dropout,                      
                     return_sequences=False))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
plot_model(model, to_file='rnn-mnist.png', show_shapes=True)

StopWatch.progress(60)
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
StopWatch.stop("compile")

# cloudmesh status=running progress=40 pid=3513


2022-07-15 10:10:35.186832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 256)           291840    
                                                                 
 lstm_1 (LSTM)               (None, 28, 256)           525312    
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 10)                2570      
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 1,345,034
Trainable params: 1,345,034
Non-trainable params: 0
_________________________________________________________________
# cloudmesh status=running progress=60 pid=351

## Train

In [4]:
StopWatch.progress(80)
StopWatch.start("train")
model.fit(x_train, y_train, epochs=1, batch_size=batch_size)
StopWatch.stop("train")

# cloudmesh status=running progress=80 pid=3513
469/469 [==============================] - 154s 321ms/step - loss: 2.1995 - accuracy: 0.2611


## Test

In [5]:
StopWatch.start("evaluate")
loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))
StopWatch.stop("evaluate")

StopWatch.benchmark()
StopWatch.progress(100)

79/79 [==============================] - 9s 106ms/step - loss: 1.9515 - accuracy: 0.3308

Test accuracy: 33.1%

+------------------+--------------------------------------------------------------------------------------------------+
| Attribute        | Value                                                                                            |
|------------------+--------------------------------------------------------------------------------------------------|
| cpu              | Intel(R) Core(TM) i7-8557U CPU @ 1.70GHz                                                         |
| cpu_cores        | 4                                                                                                |
| cpu_count        | 8                                                                                                |
| cpu_threads      | 8                                                                                                |
| date             | 2022-07-15 14:13:24.106608 

# References

1. [Advance Deep Learning with Keras](https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras)